# Azure Discovery Day 2019
## Analytics with NRT Intelligence on Azure
## Ingest, Transform, Emit

#### Summary
In this Python Jupyter notebook, you will:
1. Connect to Azure storage
2. Ingest data from CSV files in Azure storage to Spark dataframes
3. Conform and merge heterogenous data sets using the Spark dataframe API
4. Emit data to Azure storage in Parquet file format

Additionally, there are optional steps to create Hive tables on the data, query them with Spark SQL, as well as some exploratory data analysis (EDA).

In [2]:
## Need some library includes

import os

from pyspark.sql.types import *
from pyspark.sql.functions import broadcast, lit

### Variables

In [4]:
# Define some variables to minimize "hard-coding" in below cells. Note that variables could also be defined in a separate notebook for greater share-ability.

# Azure storage account information. Note that in production, we would store sensitive info in an appropriate secret store,
# such as a Databricks secret store backed by an Azure Key Vault. For a hackathon/workshop, we'll put them here for simplicity.
storage_acct_name = "PROVIDE"
storage_acct_key = "PROVIDE"
container_name = "data"

# The mount point in the DBFS file system - this will look like a local folder but points to the Azure storage location
mount_point = "/mnt/" + container_name + "/"

# Reference data files path root
path_root_ref = mount_point + "reference-data/"

# Transactional data files path root
path_root_data = mount_point + "transactional-data-small/"

# Parquet data files output root
path_root_parquet = mount_point + "parquet/"

# Parquet ref data files output root
path_root_parquet_ref = path_root_parquet + "reference/"

# Parquet trip data files output root
path_root_parquet_trips_yellow = path_root_parquet + "trips-yellow/"
path_root_parquet_trips_green = path_root_parquet + "trips-green/"
path_root_parquet_trips_all = path_root_parquet + "trips-all/"

# Set number of Parquet files to output. Simply hard-code here, could also calculate based on ... # of worker nodes/cores etc.
num_of_parquet_files = 8

### Functions

In [6]:
# Function to get a Spark DataFrame from a CSV source file

def GetDataFrameFromCsvFile(schema, path_src_file, delimiter):
  df = spark\
    .read\
    .format("csv")\
    .option("header", "true")\
    .option("delimiter", delimiter)\
    .schema(schema)\
    .load(path_src_file)
  
  return df;

In [7]:
# Given a reference dataframe (this would not make sense for large transaction dataframes), broadcast it across the cluster, lazy-cache it, and return the count, which instantiates the dataframe

def HandleReferenceDataFrame(df):
  broadcast(df)
  df.cache()
  count = df.count()
  
  return count;

In [8]:
# Delete Spark job residual files (_SUCCESS, _start*, _committed*) down the folder/file hierarchy

def CleanupSparkJobFiles(path_root):
  file_paths = GetFilesRecursive(path_root)
  
  for file_path in file_paths:
    # Get just the file name
    file_name = os.path.basename(file_path)
    # print(file_name)
    
    if file_name.startswith("_"):
      # Temp job file - delete it
      dbutils.fs.rm(file_path)
    # elif file_name.endswith(".parquet"):
      # Data file - no op
    # else:
      # Something else - no op

In [9]:
# Get iterable file list. Flattens hierarchical folder/file structure.

def GetFilesRecursive(path_root):
  final_list = []

  for directoryItem in dbutils.fs.ls(path_root):
    directoryItemPathClean = directoryItem.path.replace("%25", "%").replace("%25", "%")
    
    if directoryItem.isDir() == True:
      final_list = final_list + GetFilesRecursive(directoryItemPathClean)
    else:
      final_list.append(directoryItemPathClean)
  
  return final_list;

### Connect to Azure Storage

In [11]:
# Use the Databricks file system utilities to mount a Databricks file system location (/mnt/YOUR CONTAINER NAME) that points to the Azure storage account where data files are located
# We use variables defined above and string concatenation here so that no "hard-coding" is needed
# NOTE that this only needs to be done once as mounts survive cluster shutdown/restart.

dbutils.fs.mount(
  source = "wasbs://" + container_name + "@" + storage_acct_name + ".blob.core.windows.net",
  mount_point = mount_point,
  extra_configs = {"fs.azure.account.key." + storage_acct_name + ".blob.core.windows.net":storage_acct_key}
)

In [12]:
# This is included to remove the Azure storage mount
# Commented out since not needed for the lab, but included here "just in case" for debugging/experimenting - for example, mount, unmount, try something different, mount again

# dbutils.fs.unmount(mount_point)

In [13]:
# List contents of the Azure storage account to validate successful connect and mount
# We are using the Databricks display() function here to improve the esthetics of the output

display(dbutils.fs.ls(mount_point))

In [14]:
# Cleanup - delete Parquet output folder if it's present
# NOTE!! This will DELETE any previous Parquet that you or others have emitted to this storage location!

dbutils.fs.rm(path_root_parquet, True)

### Load Reference Data Files into DataFrames

##### Define variables to hold the source path for each of the reference data files

In [17]:
src_file_ref_payment_type = path_root_ref + "payment_type_lookup.csv"
src_file_ref_rate_code = path_root_ref + "rate_code_lookup.csv"
src_file_ref_taxi_zone = path_root_ref + "taxi_zone_lookup.csv"
src_file_ref_trip_month = path_root_ref + "trip_month_lookup.csv"
src_file_ref_trip_type = path_root_ref + "trip_type_lookup.csv"
src_file_ref_vendor = path_root_ref + "vendor_lookup.csv"

##### Define explicit schemas for each of the reference data files

We could also ingest files with schema inference (i.e. tell Spark to try to figure it out) but let's be explicit here for greater control.

In [19]:
# Payment type
schema_ref_payment_type = StructType([
    StructField("payment_type", IntegerType(), True),
    StructField("abbreviation", StringType(), True),
    StructField("description", StringType(), True)
])

# Rate code ID
schema_ref_rate_code = StructType([
    StructField("rate_code_id", IntegerType(), True),
    StructField("description", StringType(), True)
])

# Taxi zone
schema_ref_taxi_zone = StructType([
    StructField("location_id", StringType(), True),
    StructField("borough", StringType(), True),
    StructField("zone", StringType(), True),
    StructField("service_zone", StringType(), True)
])

# Trip month
schema_ref_trip_month = StructType([
    StructField("trip_month", StringType(), True),
    StructField("month_name_short", StringType(), True),
    StructField("month_name_full", StringType(), True)
])

# Trip type
schema_ref_trip_type = StructType([
    StructField("trip_type", IntegerType(), True),
    StructField("description", StringType(), True)
])

# Vendor ID
schema_ref_vendor = StructType([
    StructField("vendor_id", IntegerType(), True),
    StructField("abbreviation", StringType(), True),
    StructField("description", StringType(), True)
])

##### Load each reference data set into a Spark DataFrame

We load the data from source file into dataframe using a function (above) for that purpose.

Then we do some more optimizations for the reference dataframes:
1. Broadcast the dataframe. These are small dataframes with reference data. Broadcasting means we replicate a dataframe to each worker node in a Spark cluster, so that cross-node (cross-network) joins are avoided.
2. Lazy-cache the dataframe into memory as another performance optimization.

Last, we print the number rows in the dataframe.

In [21]:
df_ref_payment_type = GetDataFrameFromCsvFile(schema_ref_payment_type, src_file_ref_payment_type, "|")

print(HandleReferenceDataFrame(df_ref_payment_type))
display(df_ref_payment_type)

In [22]:
df_ref_rate_code = GetDataFrameFromCsvFile(schema_ref_rate_code, src_file_ref_rate_code, "|")

print(HandleReferenceDataFrame(df_ref_rate_code))
display(df_ref_rate_code)

In [23]:
df_ref_taxi_zone = GetDataFrameFromCsvFile(schema_ref_taxi_zone, src_file_ref_taxi_zone, ",")

print(HandleReferenceDataFrame(df_ref_taxi_zone))
display(df_ref_taxi_zone)

In [24]:
df_ref_trip_month = GetDataFrameFromCsvFile(schema_ref_trip_month, src_file_ref_trip_month, ",")

print(HandleReferenceDataFrame(df_ref_trip_month))
display(df_ref_trip_month)

In [25]:
df_ref_trip_type = GetDataFrameFromCsvFile(schema_ref_trip_type, src_file_ref_trip_type, "|")

print(HandleReferenceDataFrame(df_ref_trip_type))
display(df_ref_trip_type)

In [26]:
df_ref_vendor = GetDataFrameFromCsvFile(schema_ref_vendor, src_file_ref_vendor, "|")

print(HandleReferenceDataFrame(df_ref_vendor))
display(df_ref_vendor)

### Write reference data out to Parquet files

Parquet files are faster to load than CSV. They also support partitioning, but for the small reference data files, we coalesce the dataframe to 1 piece and we do not partition.

In [28]:
df_ref_payment_type.coalesce(1).write.parquet(path_root_parquet_ref + "payment-type/")

In [29]:
df_ref_rate_code.coalesce(1).write.parquet(path_root_parquet_ref + "rate-code/")

In [30]:
df_ref_taxi_zone.coalesce(1).write.parquet(path_root_parquet_ref + "taxi-zone/")

In [31]:
df_ref_trip_month.coalesce(1).write.parquet(path_root_parquet_ref + "trip-month/")

In [32]:
df_ref_trip_type.coalesce(1).write.parquet(path_root_parquet_ref + "trip-type/")

In [33]:
df_ref_vendor.coalesce(1).write.parquet(path_root_parquet_ref + "vendor/")

### Load transactions (trip data) into DataFrames

In [35]:
## Canonical ordered column list to homogenize schema - we will conform all ingested data sets to this schema

columns_rides_canonical = [
    "trip_type",
	"trip_year",
	"trip_month",
	"taxi_type",
	"vendor_id",
	"pickup_datetime",
	"dropoff_datetime",
	"passenger_count",
	"trip_distance",
	"rate_code_id",
	"store_and_fwd_flag",
	"pickup_location_id",
	"dropoff_location_id",
	"pickup_longitude",
	"pickup_latitude",
	"dropoff_longitude",
	"dropoff_latitude",
	"payment_type",
	"fare_amount",
	"extra",
	"mta_tax",
	"tip_amount",
	"tolls_amount",
	"improvement_surcharge",
    "ehail_fee",
	"total_amount"
]

In [36]:
# Define canonical schema so that we can create an empty DataFrame with the canonical schema into which we will merge all the invididual data file-ingested DataFrames

schema_rides_canonical = StructType([
    StructField("trip_type", IntegerType(), True),
    StructField("trip_year", StringType(), True),
    StructField("trip_month", StringType(), True),
    StructField("taxi_type", StringType(), True),
    StructField("vendor_id", IntegerType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("pickup_location_id", IntegerType(), True),
    StructField("dropoff_location_id", IntegerType(), True),
    StructField("pickup_longitude", StringType(), True),
    StructField("pickup_latitude", StringType(), True),
    StructField("dropoff_longitude", StringType(), True),
    StructField("dropoff_latitude", StringType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("ehail_fee", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])

#### Define a DataFrame to hold ALL rides - all years/months, both Yellow and Green

In [38]:
df_all_rides_canonical = spark.createDataFrame([], schema_rides_canonical)
df_all_rides_canonical.cache()

df_all_rides_canonical.printSchema()

#### Load taxi rides - Yellow

##### Define source file schemas - Yellow

These vary by year. We have to define several schemas to fit the different source file layouts.

In [41]:
## 2016H2, 2017, 2018
schema_rides_yellow_16H2to18 = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("RatecodeID", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])

## 2015 and 2016H1
schema_rides_yellow_15to16H1 = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("RatecodeID", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])

## 2010 though 2014
schema_rides_yellow_10to14 = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("rate_code", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("surcharge", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])

##### Schema helper functions - Yellow

In [43]:
# Function to add columns to dataframe as required to homogenize schema
# Input:  Dataframe, year and month
# Output: Dataframe with homogenized schema 

def GetSchemaHomogenizedDataframe_Yellow(source_df, trip_year, trip_month):
  years10To14 = [2010, 2011, 2012, 2013, 2014]
  
  if (trip_year >= 2017 or (trip_year == 2016 and trip_month > 6)):
    df = source_df\
      .withColumn("trip_type", lit(0))\
      .withColumn("trip_year", source_df.tpep_pickup_datetime[0:4])\
      .withColumn("trip_month", source_df.tpep_pickup_datetime[6:2])\
      .withColumn("taxi_type", lit("yellow"))\
      .withColumnRenamed("VendorID", "vendor_id")\
      .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")\
      .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")\
      .withColumnRenamed("RatecodeID", "rate_code_id")\
      .withColumnRenamed("PULocationID", "pickup_location_id")\
      .withColumnRenamed("DOLocationID", "dropoff_location_id")\
      .withColumn("pickup_longitude", lit(""))\
      .withColumn("pickup_latitude", lit(""))\
      .withColumn("dropoff_longitude", lit(""))\
      .withColumn("dropoff_latitude", lit(""))\
      .withColumn("ehail_fee", lit(0.0))

      # .withColumn("temp_payment_type", source_df.payment_type.cast(StringType()))\
      # .drop("payment_type")\
      # .withColumnRenamed("temp_payment_type", "payment_type")\

      # passenger_count
      # trip_distance
      # store_and_fwd_flag
      # fare_amount
      # extra
      # mta_tax
      # tip_amount
      # tolls_amount
      # improvement_surcharge
      # total_amount
  elif ((trip_year == 2016 and trip_month <= 6) or (trip_year == 2015)):
    df = source_df\
      .withColumn("trip_type", lit(0))\
      .withColumn("trip_year", source_df.tpep_pickup_datetime[0:4])\
      .withColumn("trip_month", source_df.tpep_pickup_datetime[6:2])\
      .withColumn("taxi_type", lit("yellow"))\
      .withColumnRenamed("VendorID", "vendor_id")\
      .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")\
      .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")\
      .withColumnRenamed("RatecodeID", "rate_code_id")\
      .withColumn("pickup_location_id", lit(0).cast(IntegerType()))\
      .withColumn("dropoff_location_id", lit(0).cast(IntegerType()))\
      .withColumn("temp_pickup_longitude", source_df.pickup_longitude.cast(StringType()))\
      .drop("pickup_longitude")\
      .withColumnRenamed("temp_pickup_longitude", "pickup_longitude")\
      .withColumn("temp_pickup_latitude", source_df.pickup_latitude.cast(StringType()))\
      .drop("pickup_latitude")\
      .withColumnRenamed("temp_pickup_latitude", "pickup_latitude")\
      .withColumn("temp_dropoff_longitude", source_df.dropoff_longitude.cast(StringType()))\
      .drop("dropoff_longitude")\
      .withColumnRenamed("temp_dropoff_longitude", "dropoff_longitude")\
      .withColumn("temp_dropoff_latitude", source_df.dropoff_latitude.cast(StringType()))\
      .drop("dropoff_latitude")\
      .withColumnRenamed("temp_dropoff_latitude", "dropoff_latitude")\
      .withColumn("ehail_fee", lit(0.0))

      # .withColumn("temp_payment_type", source_df.payment_type.cast(StringType()))\
      # .drop("payment_type")\
      # .withColumnRenamed("temp_payment_type", "payment_type")\

      # passenger_count
      # trip_distance
      # store_and_fwd_flag
      # fare_amount
      # extra
      # mta_tax
      # tip_amount
      # tolls_amount
      # improvement_surcharge
      # total_amount
  elif (trip_year in years10To14):
    df = source_df\
      .withColumn("trip_type", lit(0))\
      .withColumn("trip_year", source_df.pickup_datetime[0:4])\
      .withColumn("trip_month", source_df.pickup_datetime[6:2])\
      .withColumn("taxi_type", lit("yellow"))\
      .withColumnRenamed("rate_code", "rate_code_id")\
      .withColumn("pickup_location_id", lit(0).cast(IntegerType()))\
      .withColumn("dropoff_location_id", lit(0).cast(IntegerType()))\
      .withColumn("temp_pickup_longitude", source_df.pickup_longitude.cast(StringType()))\
      .drop("pickup_longitude")\
      .withColumnRenamed("temp_pickup_longitude", "pickup_longitude")\
      .withColumn("temp_pickup_latitude", source_df.pickup_latitude.cast(StringType()))\
      .drop("pickup_latitude")\
      .withColumnRenamed("temp_pickup_latitude", "pickup_latitude")\
      .withColumn("temp_dropoff_longitude", source_df.dropoff_longitude.cast(StringType()))\
      .drop("dropoff_longitude")\
      .withColumnRenamed("temp_dropoff_longitude", "dropoff_longitude")\
      .withColumn("temp_dropoff_latitude", source_df.dropoff_latitude.cast(StringType()))\
      .drop("dropoff_latitude")\
      .withColumnRenamed("temp_dropoff_latitude", "dropoff_latitude")\
      .withColumn("temp_payment_type", source_df.payment_type.cast(StringType()))\
      .drop("payment_type")\
      .withColumnRenamed("temp_payment_type", "payment_type")\
      .withColumnRenamed("surcharge", "extra")\
      .withColumn("improvement_surcharge",lit(0).cast(DoubleType()))\
      .withColumn("ehail_fee", lit(0.0))

      # pickup_datetime
      # dropoff_datetime
      # passenger_count
      # trip_distance 
      # store_and_fwd_flag
      # payment_type
      # fare_amount
      # mta_tax
      # tip_amount
      # tolls_amount
      # total_amount

    # Yellow taxi data 2010-2014 has some ID columns that are actually the abbreviations. We need to replace those with the integer IDs.
    # Vendor ID
    df = df\
      .withColumnRenamed("vendor_id", "abbreviation")\
      .join(df_ref_vendor, "abbreviation", "outer")\
      .drop("abbreviation")\
      .drop("description")

    # Payment Type
    df = df\
      .withColumnRenamed("payment_type", "abbreviation")\
      .join(df_ref_payment_type, "abbreviation", "outer")\
      .drop("abbreviation")\
      .drop("description")

  return df;

In [44]:
# Function to return schema for a given year and month
# Input:  Year and month
# Output: StructType for applicable schema 

def GetTaxiSchema_Yellow(trip_year, trip_month):
  years10To14 = [2010, 2011, 2012, 2013, 2014]
  
  if (trip_year >= 2017 or (trip_year == 2016 and trip_month > 6)):
    schema = schema_rides_yellow_16H2to18
  elif ((trip_year == 2016 and trip_month <= 6) or trip_year == 2015):
    schema = schema_rides_yellow_15to16H1
  elif (trip_year in years10To14):
    schema = schema_rides_yellow_10to14

  return schema;

##### Load data files

1. Load each year/month data file to a dataframe
2. Persist to Parquet for that year/month
3. Append to the "all rides" DataFrame

In [46]:
for yyyy in range(2010, 2019):
  start_month = 1
  end_month = 12
  
  # The dataset goes up to June 2018 inclusive (at the time of this writing - if the dataset is expanded with all of 2018, this should be changed)
  if yyyy == 2018:
    end_month = 6

  #print("yyyy=" + str(yyyy))
  #print("start_month=" + str(start_month))
  #print("end_month=" + str(end_month))

  yyyys = str(yyyy)
  #print("yyyys=" + yyyys)

  for m in range(start_month, end_month + 1):
    ms = "{:02d}".format(m)
    #print("ms=" + ms)
    
    # Source data file path
    path_src_file = path_root_data + "year=" + yyyys + "/month=" +  ms + "/type=yellow/yellow_tripdata_" + yyyys + "-" + ms + ".csv"
    print("path_src_filepath=" + path_src_file)
    
    # Correct schema to use
    schema = GetTaxiSchema_Yellow(yyyy, m)
    
    # Read file to dataframe
    df_file = GetDataFrameFromCsvFile(schema, path_src_file, ",")
    # df_file.printSchema()
    
    # Get dataframe with conformed schema
    df_conformed = GetSchemaHomogenizedDataframe_Yellow(df_file, yyyy, m)
    # df_conformed.printSchema()
    
    # Order columns per the canonical column list/order
    df_canonical = df_conformed[columns_rides_canonical]
    df_canonical.cache()
    # df_canonical.printSchema()
    
    # Write this year/month dataframe out to Parquet
    df_canonical.coalesce(num_of_parquet_files).write.parquet(path_root_parquet_trips_yellow + yyyys + "/" + ms + "/")

    # Append this year/month to the full rides dataframe we're also building
    df_all_rides_canonical = df_all_rides_canonical.union(df_canonical)

#### Load taxi rides - Green

##### Define source file schemas - Green

These vary by year. We have to define several schemas to fit the different source file layouts.

In [49]:
# Schema for source data based on year and month

# 2016H2, 2017, 2018
schema_rides_green_16H2to18 = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("lpep_pickup_datetime", TimestampType(), True),
    StructField("lpep_dropoff_datetime", TimestampType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("RatecodeID", IntegerType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("ehail_fee", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("trip_type", IntegerType(), True)
])

# 2015 and 2016H1
schema_rides_green_15to16H1 = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("lpep_pickup_datetime", TimestampType(), True),
    StructField("Lpep_dropoff_datetime", TimestampType(), True),
    StructField("Store_and_fwd_flag", StringType(), True),
    StructField("RateCodeID", IntegerType(), True),
    StructField("Pickup_longitude", DoubleType(), True),
    StructField("Pickup_latitude", DoubleType(), True),
    StructField("Dropoff_longitude", DoubleType(), True),
    StructField("Dropoff_latitude", DoubleType(), True),
    StructField("Passenger_count", IntegerType(), True),
    StructField("Trip_distance", DoubleType(), True),
    StructField("Fare_amount", DoubleType(), True),
    StructField("Extra", DoubleType(), True),
    StructField("MTA_tax", DoubleType(), True),
    StructField("Tip_amount", DoubleType(), True),
    StructField("Tolls_amount", DoubleType(), True),
    StructField("Ehail_fee", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("Total_amount", DoubleType(), True),
    StructField("Payment_type", IntegerType(), True),
    StructField("Trip_type", IntegerType(), True)
])

# 2013 though 2014
schema_rides_green_13to14 = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("lpep_pickup_datetime", TimestampType(), True),
    StructField("Lpep_dropoff_datetime", TimestampType(), True),
    StructField("Store_and_fwd_flag", StringType(), True),
    StructField("RateCodeID", IntegerType(), True),
    StructField("Pickup_longitude", DoubleType(), True),
    StructField("Pickup_latitude", DoubleType(), True),
    StructField("Dropoff_longitude", DoubleType(), True),
    StructField("Dropoff_latitude", DoubleType(), True),
    StructField("Passenger_count", IntegerType(), True),
    StructField("Trip_distance", DoubleType(), True),
    StructField("Fare_amount", DoubleType(), True),
    StructField("Extra", DoubleType(), True),
    StructField("MTA_tax", DoubleType(), True),
    StructField("Tip_amount", DoubleType(), True),
    StructField("Tolls_amount", DoubleType(), True),
    StructField("Ehail_fee", DoubleType(), True),
    StructField("Total_amount", DoubleType(), True),
    StructField("Payment_type", IntegerType(), True),
    StructField("Trip_type", IntegerType(), True)
])


##### Schema helper functions - Green

In [51]:
# Function to add columns to dataframe as required to homogenize schema
# Input:  Dataframe, year and month
# Output: Dataframe with homogenized schema 

def GetSchemaHomogenizedDataframe_Green(source_df, trip_year, trip_month):
  years13To14 = [2013, 2014]

  if (trip_year >= 2017 or (trip_year == 2016 and trip_month > 6)):
    df = source_df\
      .withColumn("trip_year", source_df.lpep_pickup_datetime[0:4])\
      .withColumn("trip_month", source_df.lpep_pickup_datetime[6:2])\
      .withColumn("taxi_type", lit("green"))\
      .withColumnRenamed("VendorID", "vendor_id")\
      .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")\
      .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")\
      .withColumnRenamed("RatecodeID", "rate_code_id")\
      .withColumnRenamed("PULocationID", "pickup_location_id")\
      .withColumnRenamed("DOLocationID", "dropoff_location_id")\
      .withColumn("pickup_longitude", lit(""))\
      .withColumn("pickup_latitude", lit(""))\
      .withColumn("dropoff_longitude", lit(""))\
      .withColumn("dropoff_latitude", lit(""))

      # passenger_count
      # trip_distance
      # store_and_fwd_flag
      # payment_type
      # fare_amount
      # extra
      # mta_tax
      # tip_amount
      # tolls_amount
      # ehail_fee
      # improvement_surcharge
      # total_amount
      # trip_type
  elif ((trip_year == 2016 and trip_month <= 6) or (trip_year == 2015)):
    df = source_df\
      .withColumn("trip_year", source_df.lpep_pickup_datetime[0:4])\
      .withColumn("trip_month", source_df.lpep_pickup_datetime[6:2])\
      .withColumn("taxi_type", lit("green"))\
      .withColumnRenamed("VendorID", "vendor_id")\
      .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")\
      .withColumnRenamed("Lpep_dropoff_datetime", "dropoff_datetime")\
      .withColumnRenamed("Passenger_count", "passenger_count")\
      .withColumnRenamed("Trip_distance", "trip_distance")\
      .withColumnRenamed("RateCodeID", "rate_code_id")\
      .withColumnRenamed("Store_and_fwd_flag", "store_and_fwd_flag")\
      .withColumn("pickup_location_id", lit(0).cast(IntegerType()))\
      .withColumn("dropoff_location_id", lit(0).cast(IntegerType()))\
      .withColumn("temp_pickup_longitude", source_df.Pickup_longitude.cast(StringType()))\
      .drop("Pickup_longitude")\
      .withColumnRenamed("temp_pickup_longitude", "pickup_longitude")\
      .withColumn("temp_pickup_latitude", source_df.Pickup_latitude.cast(StringType()))\
      .drop("Pickup_latitude")\
      .withColumnRenamed("temp_pickup_latitude", "pickup_latitude")\
      .withColumn("temp_dropoff_longitude", source_df.Dropoff_longitude.cast(StringType()))\
      .drop("Dropoff_longitude")\
      .withColumnRenamed("temp_dropoff_longitude", "dropoff_longitude")\
      .withColumn("temp_dropoff_latitude", source_df.Dropoff_latitude.cast(StringType()))\
      .drop("Dropoff_latitude")\
      .withColumnRenamed("temp_dropoff_latitude", "dropoff_latitude")\
      .withColumnRenamed("Payment_type", "payment_type")\
      .withColumnRenamed("Fare_amount", "fare_amount")\
      .withColumnRenamed("Extra", "extra")\
      .withColumnRenamed("MTA_tax", "mta_tax")\
      .withColumnRenamed("Tip_amount", "tip_amount")\
      .withColumnRenamed("Tolls_amount", "tolls_amount")\
      .withColumnRenamed("Ehail_fee", "ehail_fee")\
      .withColumnRenamed("improvement_surcharge", "improvement_surcharge")\
      .withColumnRenamed("Total_amount", "total_amount")\
      .withColumnRenamed("Trip_type", "trip_type")
  elif (trip_year in years13To14):
    df = source_df\
      .withColumn("trip_year", source_df.lpep_pickup_datetime[0:4])\
      .withColumn("trip_month", source_df.lpep_pickup_datetime[6:2])\
      .withColumn("taxi_type", lit("green"))\
      .withColumnRenamed("VendorID", "vendor_id")\
      .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")\
      .withColumnRenamed("Lpep_dropoff_datetime", "dropoff_datetime")\
      .withColumnRenamed("Passenger_count", "passenger_count")\
      .withColumnRenamed("Trip_distance", "trip_distance")\
      .withColumnRenamed("RateCodeID", "rate_code_id")\
      .withColumnRenamed("Store_and_fwd_flag", "store_and_fwd_flag")\
      .withColumn("pickup_location_id", lit(0).cast(IntegerType()))\
      .withColumn("dropoff_location_id", lit(0).cast(IntegerType()))\
      .withColumn("temp_pickup_longitude", source_df.Pickup_longitude.cast(StringType()))\
      .drop("Pickup_longitude")\
      .withColumnRenamed("temp_pickup_longitude", "pickup_longitude")\
      .withColumn("temp_pickup_latitude", source_df.Pickup_latitude.cast(StringType()))\
      .drop("Pickup_latitude")\
      .withColumnRenamed("temp_pickup_latitude", "pickup_latitude")\
      .withColumn("temp_dropoff_longitude", source_df.Dropoff_longitude.cast(StringType()))\
      .drop("Dropoff_longitude")\
      .withColumnRenamed("temp_dropoff_longitude", "dropoff_longitude")\
      .withColumn("temp_dropoff_latitude", source_df.Dropoff_latitude.cast(StringType()))\
      .drop("Dropoff_latitude")\
      .withColumnRenamed("temp_dropoff_latitude", "dropoff_latitude")\
	  .withColumnRenamed("Payment_type", "payment_type")\
      .withColumnRenamed("Fare_amount", "fare_amount")\
      .withColumnRenamed("Extra", "extra")\
      .withColumnRenamed("MTA_tax", "mta_tax")\
      .withColumnRenamed("Tip_amount", "tip_amount")\
      .withColumnRenamed("Tolls_amount", "tolls_amount")\
      .withColumnRenamed("Ehail_fee", "ehail_fee")\
      .withColumn("improvement_surcharge",lit(0).cast(DoubleType()))\
      .withColumnRenamed("Total_amount", "total_amount")\
      .withColumnRenamed("Trip_type", "trip_type")
  
  return df;

In [52]:
# Function to return schema for a given year and month
# Input:  Year and month
# Output: StructType for applicable schema 

def GetTaxiSchema_Green(trip_year, trip_month):
  years13To14 = [2013, 2014]
  
  if (trip_year >= 2017 or (trip_year == 2016 and trip_month > 6)):
    schema = schema_rides_green_16H2to18
  elif ((trip_year == 2016 and trip_month <= 6) or trip_year == 2015):
    schema = schema_rides_green_15to16H1
  elif (trip_year in years13To14):
    schema = schema_rides_green_13to14

  return schema;

##### Load data files

1. Load each year/month data file to a dataframe
2. Persist to Parquet for that year/month
3. Append to the "all rides" DataFrame

In [54]:
for yyyy in range(2013, 2019):
  start_month = 1
  end_month = 12

  # The green dataset starts in August 2013 and goes up to June 2018 inclusive
  # (at the time of this writing - if the dataset is expanded with all of 2018, this should be changed)
  if yyyy == 2013:
    start_month = 8
  elif yyyy == 2018:
    end_month = 6

  #print("yyyy=" + str(yyyy))
  #print("start_month=" + str(start_month))
  #print("end_month=" + str(end_month))

  yyyys = str(yyyy)
  #print("yyyys=" + yyyys)

  for m in range(start_month, end_month + 1):
    ms = "{:02d}".format(m)
    #print("ms=" + ms)
    
    # Source data file path
    path_src_file = path_root_data + "year=" + yyyys + "/month=" +  ms + "/type=green/green_tripdata_" + yyyys + "-" + ms + ".csv"
    print("path_src_filepath=" + path_src_file)
    
    # Correct schema to use
    schema = GetTaxiSchema_Green(yyyy, m)
    
    # Read file to dataframe
    df_file = GetDataFrameFromCsvFile(schema, path_src_file, ",")
    # df_file.printSchema()
    
    # Get dataframe with conformed schema
    df_conformed = GetSchemaHomogenizedDataframe_Green(df_file, yyyy, m)
    # df_conformed.printSchema()
    
    # Order columns per the canonical column list/order
    df_canonical = df_conformed[columns_rides_canonical]
    df_canonical.cache()
    # df_canonical.printSchema()
    
    # Write this year/month dataframe out to Parquet
    df_canonical.coalesce(num_of_parquet_files).write.parquet(path_root_parquet_trips_green + yyyys + "/" + ms + "/")
    
    # Append this year/month to the full rides dataframe we're also building
    df_all_rides_canonical = df_all_rides_canonical.union(df_canonical)

In [55]:
# Write combined data out to Parquet

# NOTE that we are not partitioning data here, for simplicity. However, partitioning may make sense based on
# likely query patterns, if this data will be queried (e.g. from Hive SQL, see below) repeatedly and if queries are,
# for example, likely to have predicates such as specific date ranges, in which case we might partition by trip_year and # trip_month.

df_all_rides_canonical.coalesce(8).write.parquet(path_root_parquet_trips_all)

In [56]:
## Delete Spark job files recursively

CleanupSparkJobFiles(path_root_parquet)

# Discovery Day Lab - COMPLETE

At this point, you are done with the minimum needed to proceed with Azure Discovery Day 2019.
You have ingested, transformed, and emitted data sufficient for later labs/tasks.

If you like, you can proceed with the optional steps below to explore additional capabilities on Spark. However, the following steps are not required for Discovery Day.

### Hive and Spark SQL

##### We can define Hive tables over the Parquet data we wrote to storage, then query those tables with SQL

In [60]:
db_name = "discdaydb"

In [61]:
# List current Hive databases

display(spark.catalog.listDatabases())

In [62]:
# Cleanup - drop the database first, if it already exists

spark.sql("DROP DATABASE IF EXISTS " + db_name + " CASCADE")

In [63]:
# Create the database

spark.sql("CREATE DATABASE IF NOT EXISTS " + db_name)

In [64]:
# Set default database for further operations so we don't constantly have to specify this explicitly

spark.catalog.setCurrentDatabase(db_name)

In [65]:
# List tables in our database. There should not be any, since we just created this database.
# If this is run without specifying a db name, it will use the current database set above

spark.catalog.listTables(db_name)

##### Reference data tables

Clean up first by dropping if exists, then create new tables on top of the Parquet folders we wrote to above.
Then just get a count for each table's rows as a quick sanity check. Naturally you can write other SQL statements there.

In [67]:
spark.sql("DROP TABLE IF EXISTS payment_type")
spark.sql("DROP TABLE IF EXISTS rate_code")
spark.sql("DROP TABLE IF EXISTS taxi_zone")
spark.sql("DROP TABLE IF EXISTS trip_month")
spark.sql("DROP TABLE IF EXISTS trip_type")
spark.sql("DROP TABLE IF EXISTS vendor")

In [68]:
spark.sql("""
CREATE TABLE IF NOT EXISTS payment_type(
payment_type INT,
abbreviation STRING,
description STRING)
USING parquet
LOCATION '""" + path_root_parquet_ref + """payment-type/'"""
)

In [69]:
spark.sql("""
CREATE TABLE IF NOT EXISTS rate_code(
rate_code_id INT,
description STRING)
USING parquet
LOCATION '""" + path_root_parquet_ref + """rate-code/'"""
)

In [70]:
spark.sql("""
CREATE TABLE IF NOT EXISTS taxi_zone(
location_id STRING,
borough STRING,
zone STRING,
service_zone STRING)
USING parquet
LOCATION '""" + path_root_parquet_ref + """taxi-zone/'"""
)

In [71]:
spark.sql("""
CREATE TABLE IF NOT EXISTS trip_month(
trip_month STRING,
month_name_short STRING,
month_name_full STRING)
USING parquet
LOCATION '""" + path_root_parquet_ref + """trip-month/'"""
)

In [72]:
spark.sql("""
CREATE TABLE IF NOT EXISTS trip_type(
trip_type INT,
description STRING)
USING parquet
LOCATION '""" + path_root_parquet_ref + """trip-type/'"""
)

In [73]:
spark.sql("""
CREATE TABLE IF NOT EXISTS vendor(
vendor_id INT,
abbreviation STRING,
description STRING)
USING parquet
LOCATION '""" + path_root_parquet_ref + """vendor/'"""
)

In [74]:
display(spark.sql("SELECT * FROM payment_type"))

In [75]:
display(spark.sql("SELECT * FROM rate_code"))

In [76]:
display(spark.sql("SELECT * FROM taxi_zone"))

In [77]:
display(spark.sql("SELECT * FROM trip_month"))

In [78]:
display(spark.sql("SELECT * FROM trip_type"))

In [79]:
display(spark.sql("SELECT * FROM vendor"))

##### Trip table

We will use the combined (schema-conformed, merged data for both companies) data

In [81]:
spark.sql("DROP TABLE IF EXISTS trips_all")

In [82]:
spark.sql("""
  CREATE TABLE trips_all(
    trip_type INT,
    trip_year STRING,
    trip_month STRING,
    taxi_type STRING,
    vendor_id INT,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count INT,
    trip_distance DOUBLE,
    rate_code_id INT,
    store_and_fwd_flag STRING,
    pickup_location_id INT,
    dropoff_location_id INT,
    pickup_longitude STRING,
    pickup_latitude STRING,
    dropoff_longitude STRING,
    dropoff_latitude STRING,
    payment_type INT,
    fare_amount DOUBLE,
    extra DOUBLE,
    mta_tax DOUBLE,
    tip_amount DOUBLE,
    tolls_amount DOUBLE,
    improvement_surcharge DOUBLE,
    ehail_fee DOUBLE,
    total_amount DOUBLE)
  USING parquet
  LOCATION '""" + path_root_parquet_trips_all + """'"""
)

In [83]:
# Should get 99879733

display(spark.sql("SELECT * FROM trips_all LIMIT 1000"))

## Data Exploration

##### Denormalized DataFrame

Here, we join the trips table to each of the reference tables to get reference descriptions etc.

In [86]:
df_full = spark.sql("""
select  
    t.trip_type,
    t.trip_year,
    t.trip_month,
    t.taxi_type,
    t.vendor_id,
    t.pickup_datetime,
    t.dropoff_datetime,
    t.passenger_count,
    t.trip_distance,
    t.rate_code_id,
    t.store_and_fwd_flag,
    t.pickup_location_id,
    t.dropoff_location_id,
    t.pickup_longitude,
    t.pickup_latitude,
    t.dropoff_longitude,
    t.dropoff_latitude,
    t.payment_type,
    t.fare_amount,
    t.extra,
    t.mta_tax,
    t.tip_amount,
    t.tolls_amount,
    t.improvement_surcharge,
    t.ehail_fee,
    t.total_amount,
    pt.description as payment_type_description,
    rc.description as rate_code_description,
    tzpu.borough as pickup_borough,
    tzpu.zone as pickup_zone,
    tzpu.service_zone as pickup_service_zone,
    tzdo.borough as dropoff_borough,
    tzdo.zone as dropoff_zone,
    tzdo.service_zone as dropoff_service_zone,
    tm.month_name_short,
    tm.month_name_full,
    tt.description as trip_type_description,
    v.abbreviation as vendor_abbreviation,
    v.description as vendor_description,
    year(t.pickup_datetime) as pickup_year,
    month(t.pickup_datetime) as pickup_month,
    day(t.pickup_datetime) as pickup_day,
    hour(t.pickup_datetime) as pickup_hour,
    minute(t.pickup_datetime) as pickup_minute,
    second(t.pickup_datetime) as pickup_second,
    date(t.pickup_datetime) as pickup_date,
    year(t.dropoff_datetime) as dropoff_year,
    month(t.dropoff_datetime) as dropoff_month,
    day(t.dropoff_datetime) as dropoff_day,
    hour(t.dropoff_datetime) as dropoff_hour,
    minute(t.dropoff_datetime) as dropoff_minute,
    second(t.dropoff_datetime) as dropoff_second,
    date(t.dropoff_datetime) as dropoff_date
from
    trips_all t
    left outer join payment_type pt on (t.payment_type = pt.payment_type)
    left outer join rate_code rc on (t.rate_code_id = rc.rate_code_id)
    left outer join taxi_zone tzpu on (t.pickup_location_id = tzpu.location_id)
    left outer join taxi_zone tzdo on (t.dropoff_location_id = tzdo.location_id)
    left outer join trip_month tm on (t.trip_month = tm.trip_month)
    left outer join trip_type tt on (t.trip_type = tt.trip_type)
    left outer join vendor v on (t.vendor_id = v.vendor_id)"""
)

In [87]:
df_full.cache()

In [88]:
# We will likely have a few duplicates
# 99880979 for this query without distinct, 99879733 for the pre-join query above

print(df_full.count())

In [89]:
# Drop duplicate records

df_full = df_full.dropDuplicates()

In [90]:
# Count after dropping duplicates. This will take a bit (10-15 minutes), as we have close to 100 million rows.
# After dedupe, count will be 99874067, which interestingly is less than the original (pre-join) dataframe.

print(df_full.count())

In [91]:
# Print the data frame schema

df_full.printSchema()

In [92]:
# Summary statistics
# This will take a while (8-10 minutes) for ~100 million rows as it calculates for each column
# This also uncovers some interesting issues in our data! Can you spot any right away?

display(df_full.describe())

In [93]:
# We can also just get a subset of the descriptive statistics

display(df_full.describe().select("summary"))

In [94]:
# Get top rows - head(n) or take(n)

display(df_full.head(25))

In [95]:
# Dataframe explanation

df_full.explain()

In [96]:
# Quantiles in a column

df_full.approxQuantile("total_amount", [0.25, 0.5, 0.75], 0.1)

In [97]:
# Quantiles in a column

df_full.approxQuantile("trip_distance", [0.25, 0.5, 0.75], 0.1)

In [98]:
# Frequent items

display(df_full.freqItems(["pickup_hour"]))

In [99]:
# Check correlation between two fields

df_flights_full.corr("total_amount", "df_full")